# 1. Connecting to the Refinitiv Eikon Database

## 1.1 Prerequisities

In [1]:
!pip install eikon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 KB 942.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 KB 3.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
You should consider upgrading via the '/Users/chrisbang/Desktop/Quant Trader/pywarrants_lib/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install refinitiv.dataplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 KB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 KB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 2.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 KB 869.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 1.6 MB/s eta 0:00:00a 0:00:01
Using legacy 'setup.py install' for eventemitter, since package 'wheel' is not installed.
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.0.1
    Uninstalling hyperframe-6.0.1:
      Successfully uninstalled hyperframe-6.0.1
  Attempting uninstall: hpack
    Found existing installation: hpack 4.0.0
    Uninstalling hpack-4.0.0:
      Successfully uninstalled hpack-4.0.0
  Running setup.py install for eventemitter ... done
  Attempting uninstall: h2
    Found existing instal

## 1.2 Implementing the connection functions

In [3]:
import eikon as ek

In [4]:
def connect_to_eikon(app_id: str) -> bool:
    """
    Connects to the Refinitiv Eikon Data API.

    Args:
        app_id (str): Your Refinitiv Eikon App ID.

    Returns:
        bool: True if connection is successful, False otherwise.
    """
    try:
        ek.set_app_id(app_id)
        # Test the connection by fetching some basic data
        df = ek.get_data('0700.HK', ['TR.CompanyName'])[0]
        if not df.empty:
            print("Connected to Refinitiv Eikon successfully.")
            return True
        else:
            print("Connection established, but no data retrieved.")
            return False
    except ek.EikonError as e:
        print(f"Failed to connect to Refinitiv Eikon: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

# 2. Initialise Portfolio, Hedge Parameters, and Sample Data

In [3]:
from dataclasses import dataclass, field, asdict
from datetime import date, timedelta
from typing import List, Optional, Dict
import yfinance as yf
import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class Position:
    identifier: str
    quantity: int

@dataclass
class PositionSet:
    """
    Initializes a stock portfolio with a specific date and positions.

    Attributes:
        date (date): The date of the portfolio snapshot.
        positions (List[Position]): A list of Position instances representing the portfolio holdings.
    """
    date: date
    positions: List[Position] = field(default_factory=list)

@dataclass
class HedgeConstraints:
    # Assuming a simple structure for HedgeConstraints
    max_leverage: float = 100.0
    # percentage_in_cash: Optional[float] = None
    # explode_universe: bool = True
    # exclude_target_assets: bool = True
    # exclude_corporate_actions_types: Optional[List[str]] = field(default_factory=list)
    # exclude_hard_to_borrow_assets: bool = False
    # exclude_restricted_assets: bool = False
    # max_return_deviation: float = 5.0
    min_market_cap: Optional[float] = None
    max_market_cap: Optional[float] = None
    max_adv_percentage: float = 15.0
    max_weight: float = 100.0
    max_combined_delta: Optional[float] = None

@dataclass
class HedgeParameters:
    """
    Encapsulates all parameters required for initializing and managing a hedged portfolio.

    Attributes:
        initial_portfolio (PositionSet): The initial portfolio with positions and snapshot date.
        hedge_constituents (List[str]): List of asset identifiers in the hedging universe.
        constraints (HedgeConstraints): Constraints applied to the hedging strategy.
        observation_start_date (date): Start date for observations and analysis.
        sampling_period (str): Frequency of sampling data (e.g., 'Daily', 'Weekly').
        max_adv_percentage (float): Maximum percentage of average daily volume.
        max_weight (float): Maximum weight allocated to a single asset.
        max_combined_delta (Optional[float]): Maximum combined delta allowed.
    """
    initial_portfolio: PositionSet
    hedge_constituents: List[str]
    constraints: Optional[HedgeConstraints] = None
    observation_start_date: date = field(default_factory=lambda: date.today() - timedelta(days=365))
    sampling_period: str = 'Daily'

    def __str__(self):
        """
        Provides a formatted string representation of HedgeParameters with intuitive descriptions.
        """
        lines = []
        lines.append("=== Hedge Parameters ===")
        lines.append(f"Observation Start Date: {self.observation_start_date}")
        lines.append(f"Sampling Period: {self.sampling_period}")

        # Initial Portfolio Section
        portfolio_date = self.initial_portfolio.date
        portfolio_count = len(self.initial_portfolio.positions)
        lines.append(f"Initial Portfolio Snapshot Date: {portfolio_date}")
        lines.append(f"Initial Portfolio ({portfolio_count} positions):")
        for pos in self.initial_portfolio.positions:
            lines.append(f"  - {pos.identifier}: {pos.quantity}")
        lines.append("")

        # Hedge Constituents Section
        hedge_count = len(self.hedge_constituents)
        lines.append(f"Hedge Constituents ({hedge_count} assets):")
        for asset in self.hedge_constituents:
            lines.append(f"  - {asset}")
        lines.append("")

        # Constraints Section
        if self.constraints:
            lines.append("Constraints:")
            constraints_dict = asdict(self.constraints)
            for key, value in constraints_dict.items():
                # Format list types for better readability
                if isinstance(value, list):
                    value_str = ', '.join(value) if value else 'None'
                else:
                    value_str = value
                lines.append(f"  {self._format_key(key)}: {value_str}")
        else:
            lines.append("Constraints: None")

        return "\n".join(lines)

    def _format_key(self, key: str) -> str:
        """
        Formats the constraint key to a more readable form.
        """
        return key.replace('_', ' ').capitalize()

@dataclass
class SampleData:
    """
    Retrieves and stores historical data for assets specified in HedgeParameters.

    Attributes:
        parameters (HedgeParameters): The hedge parameters object containing asset information and settings.
        historical_data (Dict[str, pd.DataFrame]): A dictionary mapping asset identifiers to their historical data DataFrames.
    """
    parameters: HedgeParameters
    historical_data: Dict[str, pd.DataFrame] = field(init=False, default_factory=dict)

    def __post_init__(self):
        """
        Initializes the SampleData by fetching historical data based on the HedgeParameters.
        """
        self.retrieve_historical_data()

    def retrieve_historical_data(self):
        """
        Fetches historical data for all assets in the initial portfolio and hedge constituents.
        """
        # Extract asset identifiers from initial portfolio and hedge constituents
        initial_assets = [pos.identifier for pos in self.parameters.initial_portfolio.positions]
        hedge_assets = self.parameters.hedge_constituents
        all_assets = list(set(initial_assets + hedge_assets))  # Remove duplicates

        print(f"Fetching historical data for {len(all_assets)} assets...")

        try:
            # Define the start and end dates for historical data retrieval
            start_date = self.parameters.observation_start_date
            end_date = date.today()

            # Map sampling_period to Eikon interval
            interval_mapping = {
                'daily': 'daily',
                'weekly': 'weekly',
                'monthly': 'monthly'
            }
            
            interval = interval_mapping.get(self.parameters.sampling_period.lower(), 'daily')

            data = ek.get_timeseries(
                instruments=all_assets,
                fields=['open', 'high', 'low', 'close', 'volume'],
                start_date=start_date,
                end_date=end_date,
                interval=interval,
                adjust='all'
            )

            # Eikon returns a MultiIndex DataFrame when multiple instruments are requested
            if isinstance(data.columns, pd.MultiIndex):
                for asset in all_assets:
                    if asset in data.columns.get_level_values(0):
                        asset_data = data[asset].dropna()
                        if not asset_data.empty:
                            self.historical_data[asset] = asset_data
                            logger.info(f"Retrieved data for {asset}.")
                        else:
                            logger.warning(f"Warning: No data found for {asset}.")
                    else:
                        logger.warning(f"Warning: {asset} not found in retrieved data.")
            else:
                # If only one instrument was requested
                asset = all_assets[0]
                asset_data = data.dropna()
                if not asset_data.empty:
                    self.historical_data[asset] = asset_data
                    logger.info(f"Retrieved data for {asset}.")
                else:
                    logger.warning(f"Warning: No data found for {asset}.")

        except ek.EikonError as e:
            logger.error(f"Failed to fetch data from Refinitiv Eikon: {e}")
            
        except Exception as e:
            logger.error(f"An unexpected error occurred: {e}")

    def get_asset_data(
        self,
        scope: str = 'all',
        assets: Optional[List[str]] = None
    ) -> Dict[str, pd.DataFrame]:
        """
        Retrieves historical data based on the specified scope.

        Args:
            scope (str): The scope of data retrieval. Options:
                - 'all': Retrieve data for all assets.
                - 'initial': Retrieve data only for the initial portfolio.
                - 'hedge': Retrieve data only for the hedge constituents.
                - 'specified': Retrieve data for specific assets provided via the 'assets' parameter.
            assets (Optional[List[str]]): A list of specific asset identifiers to retrieve data for.
                This parameter is only used when scope is 'specified'.

        Returns:
            Dict[str, pd.DataFrame]: A dictionary mapping asset identifiers to their historical data DataFrames.

        Raises:
            ValueError: If invalid scope is provided or assets are not specified when scope is 'specified'.
        """
        scope = scope.lower()
        if scope not in ['all', 'initial', 'hedge', 'specified']:
            raise ValueError("Invalid scope provided. Choose from 'all', 'initial', 'hedge', or 'specified'.")

        if scope == 'all':
            return self.historical_data

        elif scope == 'initial':
            initial_assets = [pos.identifier for pos in self.parameters.initial_portfolio.positions]
            return {asset: self.historical_data[asset] for asset in initial_assets if asset in self.historical_data}

        elif scope == 'hedge':
            hedge_assets = self.parameters.hedge_constituents
            return {asset: self.historical_data[asset] for asset in hedge_assets if asset in self.historical_data}

        elif scope == 'specified':
            if not assets:
                raise ValueError("Assets must be specified when scope is set to 'specified'.")
            return {asset: self.historical_data[asset] for asset in assets if asset in self.historical_data}

    def __str__(self):
        """
        Provides a summary of the retrieved historical data.
        """
        lines = []
        lines.append("=== Sample Data Summary ===")
        lines.append(f"Total Assets Retrieved: {len(self.historical_data)}")
        lines.append("")
        for asset, data in self.historical_data.items():
            start = data.index.min().date()
            end = data.index.max().date()
            records = len(data)
            lines.append(f"{asset}: {records} records from {start} to {end}")
        return "\n".join(lines)

    def _format_key(self, key: str) -> str:
        """
        Formats the constraint key to a more readable form.
        """
        return key.replace('_', ' ').capitalize()

    def get_all_historical_data(self) -> Dict[str, pd.DataFrame]:
        """
        Deprecated: Use get_asset_data with scope='all' instead.
        """
        return self.get_asset_data(scope='all')

    def get_initial_portfolio_data(self) -> Dict[str, pd.DataFrame]:
        """
        Deprecated: Use get_asset_data with scope='initial' instead.
        """
        return self.get_asset_data(scope='initial')

    def get_hedge_constituents_data(self) -> Dict[str, pd.DataFrame]:
        """
        Deprecated: Use get_asset_data with scope='hedge' instead.
        """
        return self.get_asset_data(scope='hedge')

if __name__ == "__main__":
    # Define Positions
    positions = [
        Position(identifier="0700.HK", quantity=100),
        Position(identifier="9988.HK", quantity=200),
        Position(identifier="2318.HK", quantity=300)
    ]

    # Create PositionSet
    portfolio = PositionSet(
        date=date(2024, 11, 17),
        positions=positions
    )

    # Define Hedge Constituents
    universe = [".HSI", ".HSCE", ".HSTECH"]

    # Define HedgeParameters
    parameters = HedgeParameters(
        initial_portfolio=portfolio,
        hedge_constituents=universe,
        observation_start_date=date(2023, 11, 18),
        sampling_period='Daily'
        # constraints can be added if needed
    )

    # Print HedgeParameters
    print(parameters)
    print("\n")

    # Instantiate SampleData to retrieve historical data
    sample_data = SampleData(parameters=parameters)

    # Print SampleData Summary
    print(sample_data)
    print("\n")

    # Retrieve historical data based on different scopes

    # 1. Retrieve DataFrame for All Assets
    all_data = sample_data.get_asset_data(scope='all')
    print("=== All Assets Data ===")
    for asset, df in all_data.items():
        print(f"{asset}: {df.shape[0]} records")

    print("\n")

    # 2. Retrieve DataFrame Only for the Initial Portfolio
    initial_data = sample_data.get_asset_data(scope='initial')
    print("=== Initial Portfolio Data ===")
    for asset, df in initial_data.items():
        print(f"{asset}: {df.shape[0]} records")

    print("\n")

    # 3. Retrieve DataFrame Only for the Hedge Constituents
    hedge_data = sample_data.get_asset_data(scope='hedge')
    print("=== Hedge Constituents Data ===")
    for asset, df in hedge_data.items():
        print(f"{asset}: {df.shape[0]} records")

    print("\n")

    # 4. Retrieve DataFrame for Specified Assets (e.g., "0700.HK" and ".HSI")
    specified_assets = ["0700.HK", ".HSI"]
    specified_data = sample_data.get_asset_data(scope='specified', assets=specified_assets)
    print("=== Specified Assets Data ===")
    for asset, df in specified_data.items():
        print(f"{asset}: {df.shape[0]} records")

    print("\n")

    # Accessing historical data for a specific asset
    asset_symbol = "0700.HK"
    asset_data = sample_data.get_asset_data(scope='specified', assets=[asset_symbol])
    if asset_data.get(asset_symbol) is not None:
        print(f"--- Historical Data for {asset_symbol} ---")
        print(asset_data[asset_symbol].head())  # Display the first few rows
    else:
        print(f"No data available for {asset_symbol}.")

=== Hedge Parameters ===
Observation Start Date: 2023-11-18
Sampling Period: Daily
Initial Portfolio Snapshot Date: 2024-11-17
Initial Portfolio (3 positions):
  - 0700.HK: 100
  - 9988.HK: 200
  - 2318.HK: 300

Hedge Constituents (3 assets):
  - .HSI
  - .HSCE
  - .HSTECH

Constraints: None


Fetching historical data for 6 assets...
[*********************100%***********************]  6 of 6 completed

3 Failed downloads:
- .HSI: No timezone found, symbol may be delisted
- .HSTECH: No timezone found, symbol may be delisted
- .HSCE: No timezone found, symbol may be delisted
Retrieved data for .HSCE.
Retrieved data for 0700.HK.
Retrieved data for .HSI.
Retrieved data for 9988.HK.
Retrieved data for 2318.HK.
Retrieved data for .HSTECH.
=== Sample Data Summary ===
Total Assets Retrieved: 6

.HSCE: 243 records from 2023-11-20 to 2024-11-15
0700.HK: 243 records from 2023-11-20 to 2024-11-15
.HSI: 243 records from 2023-11-20 to 2024-11-15
9988.HK: 243 records from 2023-11-20 to 2024-11-15
231